## Create a system mixing SMIRNOFF and non-SMIRNOFF-formatted force fields

### Parametrize a molecule with the Sage force field

First, we parametrize the ligand (toluene) with the SMIRNOFF-format Sage force field through the usual route to create an OpenMM `System`.

In [1]:
import mdtraj as md
from openff.interchange import Interchange
from openmm import app

from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_data_file_path

In [2]:
toluene_pdbfile = app.PDBFile(get_data_file_path("molecules/toluene.pdb"))
toluene_topology = Molecule.from_smiles("Cc1ccccc1").to_topology()

sage = ForceField("openff-2.0.0.offxml")

toluene = Interchange.from_smirnoff(force_field=sage, topology=toluene_topology)
toluene.positions = toluene_pdbfile.positions

In [3]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.3.offxml")

# TODO: This PDB file also has water and ions, which are not yet intelligently parsed by the toolkit.
#       For now, we simply slice out the protein.
trajectory = md.load(
    get_data_file_path("systems/test_systems/T4_lysozyme_water_ions.pdb")
)

protein = trajectory.atom_slice(trajectory.topology.select("chainid 0"))
protein.save("protein.pdb")

t4_lysozyme_molecule = Molecule.from_polymer_pdb("protein.pdb")

t4_lysozyme = Interchange.from_smirnoff(
    force_field=ff14sb, topology=t4_lysozyme_molecule.to_topology()
)
t4_lysozyme

Interchange with 6 potential handlers, non-periodic topology with 2634 atoms.

In [4]:
complex_system = t4_lysozyme + toluene

/Users/mattthompson/miniconda3/envs/openff-dev/lib/python3.9/site-packages/openff/interchange/components/interchange.py:822: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  warnings.warn(
/Users/mattthompson/miniconda3/envs/openff-dev/lib/python3.9/site-packages/openff/interchange/components/interchange.py:852: UserWarning: 'other' Interchange object has handler with name Constraints not found in 'self,' but it has now been added.
  warnings.warn(


In [5]:
complex_system.to_openmm(combine_nonbonded_forces=True)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x196305cf0> >